In [ ]:
! pip install langchain langchain-openai weaviate-client ragas datasets

In [2]:
import requests
import os
from langchain.document_loaders import TextLoader


In [3]:
url="https://raw.githubusercontent.com/hwchase17/chroma-langchain/master/state_of_the_union.txt"

In [4]:
response = requests.get(url)
response.text

'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citizens blocking tanks with 

In [5]:
with open("state_of_union.txt","w") as f:
  f.write(response.text)

In [6]:
loader = TextLoader("state_of_union.txt")
documents = loader.load()

In [7]:
from langchain.text_splitter import CharacterTextSplitter

In [8]:
splitter = CharacterTextSplitter(chunk_size=500,chunk_overlap=50)

chunks = splitter.split_documents(documents)

In [9]:
chunks[0]

Document(metadata={'source': 'state_of_union.txt'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny.')

In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Weaviate

import weaviate
from weaviate.embedded import EmbeddedOptions

/usr/local/lib/python3.10/dist-packages/google/protobuf/runtime_version.py:112: UserWarning: Protobuf gencode version 5.27.2 is older than the runtime version 5.28.0 at grpc_health/v1/health.proto. Please avoid checked-in Protobuf gencode that can be obsolete.
  warnings.warn(


In [11]:
from google.colab import userdata

OPENAI_API_KEY = userdata.get("OPEN_AI_KEY")

In [12]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [13]:
client = weaviate.Client(
    embedded_options=EmbeddedOptions()
)

INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 20897


In [14]:
vector_store = Weaviate.from_documents(
    documents=chunks,
    embedding=OpenAIEmbeddings(),
    client=client,
    by_text=False
)

<ipython-input-14-3057aedc4091>:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embedding=OpenAIEmbeddings(),


In [15]:
retriever = vector_store.as_retriever()

In [16]:
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [17]:
llm = ChatOpenAI(model="gpt-3.5-turbo",temperature = 0.7)

<ipython-input-17-befc92fbd5f7>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model="gpt-3.5-turbo",temperature = 0.7)


In [18]:
template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use two sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [19]:
prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse two sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"))])

In [20]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [21]:
questions=["what did the President say about Justic Breyer?",
           "What did the President say abou Intel's CEO?",
           "What did the President say about gun violence?"
           ]

In [22]:
groud_truths=[["The president said that Justice Breyer has dedicated his life to serve the country and thanked him for his service."],
              ["The president said that Pat Gelsinger is ready to increase Intel's investment to $100 billion."],
              ["The president asked Congress to pass proven measures to reduce gun violence."]]

In [23]:
answer = []
context = []

for query in questions:
  answer.append(rag_chain.invoke(query))
  context.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

<ipython-input-23-8b7537101b00>:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  context.append([docs.page_content for docs in retriever.get_relevant_documents(query)])


In [24]:
answer

['The President thanked Justice Breyer for his service and nominated Judge Ketanji Brown Jackson to continue his legacy on the Supreme Court.',
 "The President mentioned that Intel's CEO, Pat Gelsinger, is ready to increase their investment from $20 billion to $100 billion, which would be one of the biggest investments in manufacturing in American history.",
 'The President called on Congress to pass measures to reduce gun violence, including universal background checks and banning assault weapons and high-capacity magazines.']

In [25]:
context

[['Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court.',
  'And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.',
  'A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. \n\nAnd if we are to advance liberty and justice, we need to secure the Borde

In [26]:
data = {"question":questions,"ground_truths":groud_truths,"answer":answer,"contexts":context}

In [27]:
from datasets import Dataset

In [28]:
dataset = Dataset.from_dict(data)

In [29]:
dataset

Dataset({
    features: ['question', 'ground_truths', 'answer', 'contexts'],
    num_rows: 3
})

In [30]:
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision, answer_similarity, answer_correctness

In [31]:
result = evaluate(
    dataset = dataset,
    metrics = [
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision,
        answer_similarity,
        answer_correctness
    ]
)

Evaluating:   0%|          | 0/18 [00:00<?, ?it/s]

In [32]:
result

{'faithfulness': 1.0000, 'answer_relevancy': 0.9126, 'context_recall': 1.0000, 'context_precision': 1.0000, 'answer_similarity': 0.9378, 'answer_correctness': 0.8095}

In [33]:
result.to_pandas()

question  \
0  what did the President say about Justic Breyer?   
1     What did the President say abou Intel's CEO?   
2   What did the President say about gun violence?   

                                       ground_truths  \
0  [The president said that Justice Breyer has de...   
1  [The president said that Pat Gelsinger is read...   
2  [The president asked Congress to pass proven m...   

                                              answer  \
0  The President thanked Justice Breyer for his s...   
1  The President mentioned that Intel's CEO, Pat ...   
2  The President called on Congress to pass measu...   

                                            contexts  \
0  [Tonight, I’d like to honor someone who has de...   
1  [But that’s just the beginning. \n\nIntel’s CE...   
2  [And I ask Congress to pass proven measures to...   

                                        ground_truth  faithfulness  \
0  The president said that Justice Breyer has ded...           1.0   
1  The president said that Pat Gelsinger is ready...           1.0   
2  The president asked Congress to pass proven me...           1.0   

   answer_relevancy  context_recall  context_precision  answer_similarity  \
0          0.928567             1.0                1.0           0.919497   
1          0.895266             1.0                1.0           0.955152   
2          0.913897             1.0                1.0           0.938825   

   answer_correctness  
0            0.604874  
1            0.838788  
2            0.984706

In [34]:
result.to_pandas().iloc[0,:]["question"]

'what did the President say about Justic Breyer?'

In [35]:
result.to_pandas().iloc[0,:]["answer"]

'The President thanked Justice Breyer for his service and nominated Judge Ketanji Brown Jackson to continue his legacy on the Supreme Court.'

In [36]:
result.to_pandas().iloc[0,:]["ground_truths"]

array(['The president said that Justice Breyer has dedicated his life to serve the country and thanked him for his service.'],
      dtype=object)

In [37]:
result.to_pandas().iloc[0,:]["contexts"]

array(['Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court.',
       'And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.',
       'A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. \n\nAnd if we are to advance liberty and justice, we need to s